In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split , cross_val_score , KFold , GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score
import xgboost as xgb

In [ ]:
train_df = pd.read_csv('train_new_2v.csv')
train_df.tail()

In [ ]:
train_df.index = train_df['VisitNumber']
train_df.drop(columns='VisitNumber',inplace=True)
train_df.tail()

In [ ]:
train_df.shape

In [ ]:
train_X = train_df.iloc[:,1:]
train_y = train_df.iloc[:,0]

In [ ]:
train_X.shape,train_y.shape

## Xgboost

In [15]:
dtrain = xgb.DMatrix(train_X,train_y)
dtrain

In [16]:
parameters = {'n_estimators':[ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91], 'max_depth': [ 1,  6, 11, 16]}

In [17]:
params_test = {'n_estimators': 1, 'max_depth': 1}

In [18]:
%%time
d_train1 = xgb.cv(params=params_test,  dtrain = dtrain , num_boost_round=10, nfold=10)

CPU times: user 14.4 s, sys: 1.11 s, total: 15.5 s
Wall time: 3.45 s


In [55]:
d_train1

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,258.959375,0.408863,258.931838,4.002206
1,237.680652,0.398662,237.657417,3.778654
2,226.524615,0.405316,226.504861,3.659643
3,220.852049,0.412314,220.834538,3.620617
4,216.827116,0.388797,216.812756,3.391418
5,214.766527,0.395543,214.752982,3.426382
6,213.083183,0.388326,213.070245,3.390685
7,211.674711,0.378963,211.662767,3.313813
8,210.485812,0.384279,210.472821,3.369230
9,209.630467,0.377877,209.618419,3.334145


In [58]:
%%time
d_train = []
for i in parameters['n_estimators']:
    for j in parameters['max_depth']:
        params = {'n_estimators':i,'max_depth':j}
        d_train.append(xgb.cv(params=params,dtrain = dtrain,num_boost_round=10,nfold=5))

CPU times: user 22min 50s, sys: 23.3 s, total: 23min 14s
Wall time: 3min 22s


In [59]:
d_train

[   train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
 0       258.958643        0.576271      258.948660       2.837909
 1       237.679486        0.529881      237.675973       2.621483
 2       226.523062        0.525473      226.525970       2.412423
 3       220.850226        0.529723      220.858130       2.267453
 4       216.825760        0.475810      216.836108       2.009091
 5       214.764938        0.485488      214.778280       1.960330
 6       213.081607        0.478167      213.095462       1.933071
 7       211.673138        0.456150      211.687790       1.836320
 8       210.484494        0.458549      210.499564       1.846668
 9       209.629101        0.462011      209.644589       1.844180,
    train-rmse-mean  train-rmse-std  test-rmse-mean  test-rmse-std
 0       241.337466        0.591930      241.493414       2.681505
 1       207.757376        0.577529      208.059711       2.490490
 2       188.329468        0.587210      188.778940       2.4

In [30]:
%%time
d_train_ls = []
for i in parameters['n_estimators']:
    for j in parameters['max_depth']:
        params = {'n_estimators':i,'max_depth':j}
        d_train = xgb.cv(params=params,dtrain = dtrain,num_boost_round=10,nfold=5)
        d_train_ls.append(np.mean(d_train)[2])

CPU times: user 22min 35s, sys: 24.1 s, total: 22min 59s
Wall time: 3min 21s


In [45]:
sorted(d_train_ls)

[169.60737454000002,
 169.60737454000002,
 169.60737454000002,
 169.60737454000002,
 169.60737454000002,
 169.60737454000002,
 169.60737454000002,
 169.60737454000002,
 169.60737454000002,
 169.60737454000002,
 173.00759732,
 173.00759732,
 173.00759732,
 173.00759732,
 173.00759732,
 173.00759732,
 173.00759732,
 173.00759732,
 173.00759732,
 173.00759732,
 180.66893310000003,
 180.66893310000003,
 180.66893310000003,
 180.66893310000003,
 180.66893310000003,
 180.66893310000003,
 180.66893310000003,
 180.66893310000003,
 180.66893310000003,
 180.66893310000003,
 222.05505274,
 222.05505274,
 222.05505274,
 222.05505274,
 222.05505274,
 222.05505274,
 222.05505274,
 222.05505274,
 222.05505274,
 222.05505274]

In [36]:
ls =[]
ls1 = [ 1, 11, 21, 31, 41, 51, 61, 71, 81, 91]
ls2 = [ 1,  6, 11, 16]
for i in ls1:
    for j in ls2:
        ls.append((i,j))

In [57]:
d_train_df = pd.DataFrame()
d_train_df['score'] = d_train_ls
d_train_df.index = ls
d_train_df.sort_values(by='score').head()
# In index, first component is about n_estimators second component is about max_depth

,score
"(41, 16)",169.607375
"(81, 16)",169.607375
"(71, 16)",169.607375
"(61, 16)",169.607375
"(51, 16)",169.607375


In [ ]:
xgb_model = xgb.XGBClassifier(max_depth=16,n_estimators=41)

In [ ]:
xgb_model = xgb_model.fit(train_X,train_y)

In [ ]:
confusion_matrix(train_y,xgb_model.predict(train_X))

In [ ]:
print(classification_report(train_y,xgb_model.predict(train_X)))